# やること

- マルチスレッド化
- tmpファイル生成 -> 削除

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(100)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = (1,1,1,1)

In [2]:
a_th=0.01
threads = 8
tmp_root = 'tmp'

path_methyl_diff = '../methyl_diff/methyl_diff'

In [3]:
f_pos = '../test/posi.txt'
f_neg = '../test/nega.txt'

In [4]:
df = pd.read_csv(f_pos, sep="\t", header=None)
df.columns=['chr', 'start', 'end', 'P_me', 'P_de', 'P_low', 'P_ratio', 'P_high']

df2= pd.read_csv(f_neg, sep="\t", header=None)
df2.columns=['chr', 'start', 'end', 'N_me', 'N_de', 'N_low', 'N_ratio', 'N_high']

df3= pd.merge(df, df2, on=["chr", "start", "end"])

In [5]:
df3.head()

,chr,start,end,P_me,P_de,P_low,P_ratio,P_high,N_me,N_de,N_low,N_ratio,N_high
0,chrX,170531501,170533500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95
1,chrX,170532001,170534000,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95
2,chrX,170532501,170534500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95
3,chrX,170533001,170535000,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95
4,chrX,170533501,170535500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95


In [6]:
# 別にいらないが。
df_meth = pd.read_csv('../methyl_diff/diff.in', sep=' ', header=None)
df_meth.columns = ['P_me', 'P_de', 'N_me', 'N_de']
df_meth.head()

,P_me,P_de,N_me,N_de
0,0,18,0,9
1,0,3,0,4
2,0,20,0,14
3,0,18,0,19
4,0,20,0,15


In [7]:
index_split = np.array_split(df3.index, threads)

In [8]:
import subprocess
from io import StringIO
import os

os.makedirs(tmp_root, exist_ok=True)

procs=[]
for i in range(threads):
    proc=subprocess.Popen(path_methyl_diff, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    procs.append(proc)


list_out = []
for i,proc in enumerate(procs):
    df3.loc[index_split[i], ['P_me', 'P_de', 'N_me', 'N_de']].to_csv(
        tmp_root+'/tmp_meth.{}.txt'.format(i), sep=' ', header=None, index=None)
    data = open(tmp_root+'/tmp_meth.{}.txt'.format(i), 'r').read()
    stdout_data, stderr_data = proc.communicate(data.encode('utf-8'))

    list_out.append(pd.read_csv(StringIO(stdout_data.decode('utf-8')), header=None))
    os.remove(tmp_root+'/tmp_meth.{}.txt'.format(i))
    
if os.listdir(tmp_root) == []:
    os.rmdir(tmp_root)
    
df3['Pvalue_PN'] = list(pd.concat(list_out).reset_index(drop=True)[0])
df3['Pvalue_NP'] = 1 - df3['Pvalue_PN']

flag =[]
thresh = a_th/df3.shape[0]

for pos,row in df3.iterrows():
    if row["Pvalue_PN"] < thresh:
        flag.append(1)
    elif row["Pvalue_NP"] < thresh:
        flag.append(2)
    else:
        flag.append(0)

df3["flag"] = flag

In [9]:
df3.head()

,chr,start,end,P_me,P_de,P_low,P_ratio,P_high,N_me,N_de,N_low,N_ratio,N_high,Pvalue_PN,Pvalue_NP,flag
0,chrX,170531501,170533500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5,0.5,0
1,chrX,170532001,170534000,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5,0.5,0
2,chrX,170532501,170534500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5,0.5,0
3,chrX,170533001,170535000,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5,0.5,0
4,chrX,170533501,170535500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5,0.5,0


In [10]:
df3[df3['flag'] == 1]

,chr,start,end,P_me,P_de,P_low,P_ratio,P_high,N_me,N_de,N_low,N_ratio,N_high,Pvalue_PN,Pvalue_NP,flag
281,chrX,170672001,170674000,11012,395,0.962,0.965,0.968,431,0,0.993,0.998,1.000,3.137100e-07,1.000000,1
289,chrX,170676001,170678000,23072,828,0.963,0.965,0.967,17553,456,0.973,0.975,0.977,1.653580e-08,1.000000,1
290,chrX,170676501,170678500,21754,832,0.961,0.963,0.965,15120,427,0.970,0.972,0.975,1.928720e-07,1.000000,1
292,chrX,170677501,170679500,10860,453,0.957,0.960,0.963,3753,80,0.975,0.979,0.983,2.714580e-09,1.000000,1
405,chrX,170734001,170736000,63,25,0.630,0.713,0.787,5405,707,0.877,0.884,0.891,6.910130e-06,0.999993,1
407,chrX,170735001,170737000,194,112,0.588,0.633,0.678,7642,1537,0.826,0.833,0.839,1.349910e-12,1.000000,1
408,chrX,170735501,170737500,216,111,0.616,0.660,0.702,8120,1918,0.802,0.809,0.815,2.058830e-10,1.000000,1
410,chrX,170736501,170738500,23,89,0.151,0.209,0.276,1112,791,0.566,0.584,0.603,-5.488220e-13,1.000000,1
559,chrX,170811001,170813000,272,78,0.738,0.776,0.811,270,12,0.932,0.955,0.973,6.456040e-12,1.000000,1
616,chrX,170839501,170841500,0,20,0.002,0.032,0.133,480,12,0.961,0.974,0.984,5.730970e-15,1.000000,1
